In [145]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [146]:
import mubind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = '../../annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

Using device: cuda:0


In [147]:
import itertools

def _mismatch(word, letters, num_mismatches):
    for locs in itertools.combinations(range(len(word)), num_mismatches):
        this_word = [[char] for char in word]
        for loc in locs:
            orig_char = word[loc]
            this_word[loc] = [l for l in letters if l != orig_char]
        for poss in itertools.product(*this_word):
            yield "".join(poss)



In [148]:
# motif = "ATT"
# x2, y2 = mb.datasets.simulate_xy(motif, n_trials=2100, seqlen=len(motif) + 1, max_mismatches=min(len(motif), 2), counts_size=1)
# x2, y2
# df = pd.DataFrame(data=y2)
# df.columns = [1]
# df.index = x2
# df[0] = 0
# df = df[[0, 1]]

In [149]:
df = mb.bindome.datasets.ProBound.ctcf(flank_length=0)
df = df.sort_values(1, ascending=False).reset_index(drop=True)
# data = df.head(1000)
data = df.copy()
# data = df.sample(1500, random_state=50)
# data = df.copy()
data.index = data['seq']

# remove as many nucleotides as posisble (faster training/convergence)
data.index = data.index.astype(str).str[45:]


# data.index = data.index.astype(str).str[-15:]
del data['seq']

In [150]:
data.shape

(120096, 2)

In [151]:
data = data.sample(data.shape[0], random_state=100)

In [152]:
# # define batches
# batch = np.random.randint(2, size=data.shape[0])

In [153]:
# # data[batch == 0] = data[batch == 0] + 20
# # data[batch == 1] = data[batch == 1] + 10
# data['batch'] = batch
#
# data = data[data['batch'] == 1]

In [154]:


from matplotlib import rcParams
rcParams['figure.figsize'] = 5, 1

In [155]:
# TODO: skipped for now
n_rounds = 1

print('loading object (# entries)', data.shape[0])
dataset = mb.datasets.SelexDataset(data, n_rounds=n_rounds, labels=[0, 1])
train = tdata.DataLoader(dataset=dataset,
                         # batch_size=256,
                         batch_size=512,
                         shuffle=False)

loading object (# entries) 120096
(120096, 2)


In [156]:
import warnings
warnings.filterwarnings('ignore')

In [157]:
import numpy as np

In [158]:
%load_ext line_profiler


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [159]:
# %lprun -f mb.tl.train_network mb.tl.train_network(model, train, device, next_optimiser, criterion, num_epochs=20, early_stopping=100, log_each=2, dirichlet_regularization=0, exp_max=40, verbose=0)

In [160]:
# %lprun -f mb.tl.train_iterative mb.tl.train_iterative(train, device, w=18, show_logo=False, opt_kernel_shift=0, opt_kernel_length=0, dirichlet_regularization=dirichlet_regularization, lr=[0.01, 0.01], weight_decay=[0.01, 0.001], ignore_kernel=ignore_kernel, num_epochs=2, early_stopping=100, use_dinuc=False, n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None)

In [161]:
y_pred = torch.tensor(data.copy().to_numpy() - 100)
y_true = torch.tensor(data.copy().to_numpy() - 100)
torch.mean(y_pred - y_true * torch.log(y_pred))
torch.log(y_pred)
criterion = mb.tl.PoissonLoss()
criterion(y_pred, y_true)

tensor(nan)

In [ ]:
# assign batch and data type

# data['batch'] = 0
# data['is_count_data'] = 1
# n_batches=3

dirichlet_regularization = 0
# for dirichlet_regularization_log in range(-5, 3):

n_epochs = 100
log_each = 2
n_kernels = 2
w = 15 # min(len(motif), 6)
opt_kernel_shift=1
opt_kernel_length=1

from matplotlib import rcParams
rcParams['figure.figsize'] = 8, 1

import time
t0 = time.time()

criterion = mb.tl.PoissonLoss()

for ignore_kernel in [1]: # [0, 1]:
    print('\n\n##NEW JOB')
    print('ignore kernel', ignore_kernel)
    model_by_k, res_next = mb.tl.train_iterative(train, device, w=w, show_logo=0, criterion=criterion,
                                                 opt_kernel_shift=0, opt_kernel_length=0,
                                                 dirichlet_regularization=dirichlet_regularization, # 10 ** dirichlet_regularization_log,
                                                 lr=[0.01, 0.01], weight_decay=[0.01, 0.001], ignore_kernel=ignore_kernel,
                                                 num_epochs=n_epochs, early_stopping=[10, 100], use_dinuc=False, # optimiser=torch.optim.LBFGS,
                                                 n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None) #  seed=seed) # seeds.index[0]) #
    print('##DONE....\n\n')


print('total time: %.3f s' % ((time.time() - t0)))
# res = []
# model_by_k = {}

model = model_by_k





##NEW JOB
ignore kernel 1
next w 15 <class 'int'>
# rounds 1
# batches 1
# enr_series True

Kernel to optimize 0

FREEZING KERNELS
setting grad status of kernel at 0 to 1
setting grad status of kernel at 1 to 0


kernels mask [0 1]
optimizing using <class 'torch.optim.adam.Adam'> and <class 'mubind.tl.loss.PoissonLoss'> n_epochs 100 early_stopping 10
lr= 0.01, weight_decay= 0.01, dir weight= 0
Epoch:  3, Loss: 0.843945 , best epoch: 1 secs per epoch: 18.697 s
Epoch:  5, Loss: 0.843948 , best epoch: 1 secs per epoch: 17.334 s
Epoch:  7, Loss: 0.843949 , best epoch: 1 secs per epoch: 16.854 s
Epoch:  9, Loss: 0.843949 , best epoch: 1 secs per epoch: 16.588 s
Epoch: 11, Loss: 0.843949 , best epoch: 1 secs per epoch: 16.774 s
Epoch: 12, Loss: 0.8439 , best epoch: 1 secs per epoch: 18.146 s
early stop!
total time: 199.60410070419312s 12
secs per epoch: 18.146 s

Kernel to optimize 1

FREEZING KERNELS
setting grad status of kernel at 0 to 0
setting grad status of kernel at 1 to 1


kernels

### We can visualize the overall results obtained by the network, once the training is finished

In [ ]:
import mubind as mb

In [ ]:
model = model_by_k
mb.tl.scores(model, train)


In [ ]:
counts = mb.tl.predict(model, train)
counts

In [ ]:
mb.pl.kmer_enrichment(model, train, log_scale=False, style='scatter', ylab='t1', xlab='p1')


In [ ]:
mb.pl.conv_mono(model)
